# Report

## Introduction and data

> REMOVE THE FOLLOWING TEXT

This section includes an introduction to the project motivation, data, and (research) question.
### Introduction

Durch die Digitalisierung der Versicherungsbranche wurde eine neue Norm gesetzt, die es ermöglichte besser auf die Bedürfnisse der Kunden einzugehen. Die Versicherungen stellten sich damit einer bereits bekannten Herausforderung, dem Versicherungsbetrug. Jede zehnte Schadensmeldung ist betrugsverdächtig und kostet den deutschen Versicherungen rund fünf Milliarden Euro pro Jahr (GDV, 2020). Um Versicherungsbetrug zu erkennen, wird spezielle Betrugserkennungssoftware verwendet. Hierbei nutzen Versicherungsbetrüger zunehmend Anleitungen aus dem Internet, die es erleichtern die Betrugserkennungssoftware zu umgehen (GDV, 2020). 

*Quelle: https://www.gdv.de/gdv/medien/medieninformationen/sorge-der-versicherer-corona-gibt-betruegern-auftrieb-61842*

Mit diesem Datensatz untersuchen wir, ob und welche Maßnahmen nötig sind, um den Prozess der Erkennung von Versicherungsbetrug zu verbessern. 

### Forschungsfrage

Welche Faktoren sind für die Erkennung von Betrugsfällen relevant?

Wir haben folgende Hypothesen:

1. Unter 35 jährige Versicherungskunden betrügen öfters.-> Relevante Variablen: InsuredAge, ReportedFraud

2. Männer betrügen häufiger als Frauen.-> Relevante Variablen: InsuredGender, ReportedFraud

3. Schadensfälle ohne polizeilicher Dokumentation sind häufiger Betrugsfälle.-> Relevante Variablen: AuthoritiesContacted, ReportedFraud




### Data Dictionary

Beschreibung der relevanten Variablen

- InsuredAge: Das Alter des Kunden zum Zeitpunkt des Anspruchs. (Datentyp: Integer)<br>
- InsuredGender: Das Geschlecht des Kunden. (Datentyp: String)<br>
- AuthorotiesContacted: Information darüber, ob Behörden (Polizei, Krankenwagen, Feuerwehr oder andere) kontaktiert wurden. (Datentyp: String)<br>
- ReportedFraud: Information darüber, ob der Anspruch als Betrug gemeldet wurde oder nicht. (Datentyp: String)<br>

<br>


| Name  |    Type 	  |    Format 	  |
|---	|---	          	|---	          	|
| AuthoritiesContacted  	|   nominal     | object      	    | 
| InsuredAge  	|   numerical	       	    | int |
| InsuredGender  	|   nominal	       	    | object |
| ReportedFraud  	|   	 nominal      	| object|


<br>


### Data Corrections

> Use content from the [BIG IDEA worksheet](https://docs.google.com/document/d/1-GZvhdbhLYLB_Bo1arj1rgTqbJ5SUoU21vtgbYEhVqk/edit?usp=sharing)   

Describe the data and definitions of key variables.

It should also include some exploratory data analysis.

*All of the EDA won't fit in the paper, so focus on the EDA for the response variable and a few other interesting variables and relationships.*

## Setup

In [ ]:
import pandas as pd
import altair as alt
import numpy as np
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
alt.data_transformers.disable_max_rows()

## Data

### Import Data

In [ ]:
ROOT= "https://raw.githubusercontent.com/christophersegatz/dst-project/main/data/insurance_fraud/"
DATA = "fraud.csv"

df=pd.read_csv(ROOT + DATA)

### Data Structure

In [ ]:
df.info()

### Data Corrections

In [ ]:
# Nicht benötigte Tabellen aus dem Dataframe entfernen
df.drop(["DateOfIncident","TypeOfCollission", "SeverityOfIncident", "IncidentState", "IncidentCity", "IncidentAddress", "IncidentTime",
         "NumberOfVehicles", "PropertyDamage", "BodilyInjuries", "Witnesses", "PoliceReport", "AmountOfTotalClaim", "AmountOfInjuryClaim",
        "AmountOfPropertyClaim", "AmountOfVehicleDamage", "InsuredZipCode", "InsuredOccupation", "InsuredHobbies",
        "CapitalGains", "CapitalLoss", "Country", "InsurancePolicyNumber", "DateOfPolicyCoverage", "InsurancePolicyState", "Policy_CombinedSingleLimit",
        "Policy_Deductible", "PolicyAnnualPremium", "UmbrellaLimit", "InsuredRelationship", "CustomerID", "TypeOfIncident",
        "InsuredEducationLevel", "CustomerLoyaltyPeriod"], axis= 1, inplace=True)

# Entfernt alle Zeilen, die mindestens einen Null-Wert haben
df.dropna()

In [ ]:
# Als kategoriale Variablen abändern
df = df.astype(
    {
        "AuthoritiesContacted": "category",
        "InsuredGender": "category",
})

## Analyse

### Deskriptive Analyse

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
mean_insured_age = df['InsuredAge'].mean()
mean_insured_age = round(mean_insured_age, 0)
print("Durchschnittliches Alter der Versicherten: {:.0f}".format(mean_insured_age))


# Boxplot-Diagramm
box = alt.Chart(df).mark_boxplot().encode(
    x=alt.X('InsuredAge:Q',axis=alt.Axis(title=""))
)

hist = alt.Chart(df).mark_bar().encode(
    alt.X("InsuredAge", bin=alt.Bin(maxbins=30), title="Alter"),
    y=alt.Y('count()', axis=alt.Axis(title="Anzahl der Versicherten")),
)

hist | box

Erkenntnis aus den Graphen:

- Das maximale Alter der Versicherten beträgt 64 und das minimalste 19.
- Der dritte Quartil (Q3) des Alters liegt bei 44, was bedeutet, dass 75% der Versicherten jünger als 44 Jahre alt sind.
- Der erste Quartil (Q1) des Alters liegt bei 33, was bedeutet, dass 25% der Versicherten jünger als 33 Jahre alt sind.
- Der Median des Alters beträgt 38, was bedeutet, dass 50% der Versicherten jünger und 50% älter als 38 Jahre alt sind.

In [ ]:
# Fehlende werte entfernen
df_gender = df.dropna(subset=['InsuredGender'])

bar_gender = alt.Chart(df_gender).mark_bar(size=40).encode(
    alt.X("InsuredGender:N", axis=alt.Axis(title=""), sort=['F', 'M']),
    alt.Y("count()", axis=alt.Axis(title="")),
    color=alt.Color('InsuredGender:N', legend=alt.Legend(title="Geschlecht"))
)

text_gender = alt.Chart(df_gender).mark_text(
    align='center',
    baseline='middle',
    fontWeight='bold',
    dy=-10
).encode(
    x=alt.X('InsuredGender:N', axis=alt.Axis(title="")),
    y=alt.Y('count()', axis=alt.Axis(title='Anzahl Versicherter')),
    text=alt.Text('count()'),
)
gender_layer = bar_gender+text_gender

gender_layer.configure_axis(
    labelAngle=0
).properties(
    width=400,
    height=200,
    title="Anzahl Versicherter nach Geschlecht"
).configure_legend(
    orient='right',
    strokeWidth=1,
    padding=10,
    strokeColor='grey'
)

Die Anzahl der weiblichen (15644) ist höher als die, der männlichen Versicherten (13162). 

In [ ]:
# Barchart
bar_auth = alt.Chart(df).mark_bar().encode(
    alt.X('AuthoritiesContacted:N', axis=alt.Axis(title=''),),
    alt.Y('count()', axis=alt.Axis(title='')),
    alt.Color('AuthoritiesContacted:N', legend=alt.Legend(title='Kontaktierte Behörden')),
)
text_auth = alt.Chart(df_gender).mark_text(
    align='center',
    baseline='middle',
    fontWeight='bold',
    dy=-10
).encode(
    x=alt.X('AuthoritiesContacted:N', axis=alt.Axis(title=''),),
    y=alt.Y('count()', axis=alt.Axis(title='Anzahl')),
    text=alt.Text('count()')
)
grouped_auth=bar_auth+text_auth

grouped_auth.properties(
    title='Relative Häufigkeit von kontaktierten Behörden',
    height=200,
    width=400
).configure_legend(
    orient='right',
    strokeWidth=1,
    padding=10,
    strokeColor='grey'
).configure_axis(
    labelAngle=0
)

Bei den meisten Fällen wurde die Polizei (8313) kontaktiert. Ebenso gab es viele Fälle mit Kontakt zur Feuerwehr (6513) und Krankenwagen(5726).
Bei 5564 Fällen wurden andere kontakiert und bei 2690 Fällen gar keiner.

In [ ]:
bar_fraud = alt.Chart(df).mark_bar(size=40).encode(
    alt.X('ReportedFraud:N', axis=alt.Axis(title=''),),
    alt.Y('count()', axis=alt.Axis(title='')),
    alt.Color('ReportedFraud:N', legend=alt.Legend(title='Gemeldeter Betrug')),
)

text_fraud = alt.Chart(df_gender).mark_text(
    align='center',
    baseline='middle',
    fontWeight='bold',
    dy=-10
).encode(
    x=alt.X('ReportedFraud:N', axis=alt.Axis(title=''),),
    y=alt.Y('count()', axis=alt.Axis(title='Anzahl')),
    text=alt.Text('count()')
)
group_fraud = bar_fraud + text_fraud

group_fraud.properties(
    title='Relative Häufigkeit von gemeldeten Betrugsfällen',
    height=200,
    width=400
).configure_legend(
    orient='top-right',
    strokeWidth=1,
    padding=10,
    strokeColor='grey'
).configure_axis(
    labelAngle=0
)

Von 28806 Fällen sind 7780 als Betrug gemeldet worden. 21026 Fälle wurden nicht als Betrug gemeldet.

### Explorative Analyse

<b> H1: Unter 35 jährige Versicherungskunden betrügen öfters. </b> </br> </br> 
Diese Hypothese wurde gewählt, um zu untersuchen, ob die Altersgruppe der Versicherungskunden einen Einfluss auf die Betrugsrate hat.

In [ ]:

pd.crosstab(index=df['is_over_35'], 
            columns=df['ReportedFraud'],
            normalize='index').round(4) * 100

Die Betrugsrate bei Kunden unter 35 Jahren ist nur um 0.9 Prozentpunkte höher als bei Kunden über 35 Jahren. </br>
Daher ist die Hypothese nur bedingt bestätigt.

<b>H2: Männer betrügen häufiger als Frauen.</b> </br></br>
Diese Hypothese wurde gewählt, da Statistiken zeigen das Männer häufiger Straftaten begehen als Frauen. </br>
<i>Quelle: Statistisches Bundesamt. (29. November, 2022). Anzahl der rechtskräftig verurteilten Personen in Deutschland nach Geschlecht von 2011 bis 2021 [Graph]. In Statista. Zugriff am 01. Februar 2023, von https://de.statista.com/statistik/daten/studie/1068769/umfrage/rechtskraeftig-verurteilte-personen-in-deutschland-nach-geschlecht/ </i> </br></br>
Es soll daher untersucht werden ob mehr Männer einen Versicherungsbetrug versuchen.</br>

In [ ]:
pd.crosstab(index=df['InsuredGender'], 
            columns=df['ReportedFraud'],
            normalize='index').round(4) * 100

Bei den Männern betrügen 28.25% - bei den Frauen 25.97%.
Die Hypothese ist somit bestätigt. 

In [ ]:
pd.crosstab(index=df['AuthoritiesContacted'], 
            columns=df['ReportedFraud'],
            normalize='index').round(4) * 100


Wurde keine Authority kontaktiert ist die Betrugsrtate mit 9.88% am geringsten. Wurde die Polizei kontaktiert ist die Betrugsrate mit 24.46% am 2. höchsten. 

## Visualizations

> REMOVE THE FOLLOWING TEXT

This section includes a brief description of your visualization creation process.

Explain the reasoning for the type of visualization you're using and what other types you considered. 

Additionally, show how you arrived at the final visualization by describing the plot selection process, variable transformations (if needed), and any other relevant considerations that were part of the visualization creation process.

## Conclusion + recommended action


> REMOVE THE FOLLOWING TEXT

In this section you'll include a summary of what you have learned about your (research) question along with (statistical) arguments supporting your conclusions.

In addition, discuss the limitations of your analysis and provide suggestions on ways the analysis could be improved.

Any potential issues pertaining to the reliability and validity of your data and appropriateness of the statistical analysis should also be discussed here.

Lastly, this section will include your recommended action.